# Import des librairies

In [18]:
import pandas as pd
import sys
sys.path.append('..')
from scripts.Cleaning import Featurescleaning
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.decomposition import PCA


# Récupération des données avec features et pré-nettoyage

In [19]:
df = pd.read_csv('../data/all_data_for_10000_lines.csv')
df = Featurescleaning(df)
df

,tconst,titletype,primarytitle,isadult,startyear,genres,averagerating,numvotes,seasonnumber,episodenumber,regionnumber,regionlist,actor,self,producer,actress,director,weighted_score
0,tt12605172,tvEpisode,episode 12,False,2020.0,"Action,Mystery",NaN,NaN,1.0,2.0,8,"[PT, IN, FR, IT, DE, JP, ES]","[Tatsuya Fujiwara, Renji Ishibashi, Masanobu A...",[],[],[Mikako Tabe],[Eiichirô Hasumi],6.949817
1,tt12605176,tvSpecial,shaheb bibi golam,False,2016.0,Drama,NaN,NaN,NaN,NaN,2,[BD],[],[],[],[],[],6.949817
2,tt12605178,short,girl in the mirror,False,2020.0,"Music,Short",NaN,NaN,NaN,NaN,2,[US],[],[],[],[],[],6.949817
3,tt1260518,tvEpisode,loose ends,False,2008.0,Drama,NaN,NaN,18.0,5.0,1,[],[],[],[],[],[],6.949817
4,tt12605180,tvEpisode,episode 13,False,2020.0,"Action,Mystery",NaN,NaN,1.0,3.0,8,"[PT, IN, FR, IT, DE, JP, ES]","[Tatsuya Fujiwara, Renji Ishibashi, Masanobu A...",[],[],[Mikako Tabe],[Eiichirô Hasumi],6.949817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt12624184,tvEpisode,cancin de cuna,False,2001.0,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,2,[MX],[],[],[],[],"[Felipe Nájera, Francisco Franco Alba]",6.949817
9996,tt12624186,videoGame,go diego go great dinosaur rescue,False,2008.0,Adventure,5.3,7.0,NaN,NaN,2,[US],[],[],[],[],[],6.945251
9997,tt1262419,movie,sanpei the fisher boy,False,2009.0,Drama,6.1,109.0,NaN,NaN,5,"[XWW, BR, JP, CN]",[],[],[],[],[],6.872558
9998,tt12624194,movie,debbies time,True,1971.0,Adult,NaN,NaN,NaN,NaN,2,[US],[],[],[Sven Peters],[],[Sven Peters],6.949817


# Préparation de la pipeline pour la prédiction de la popularité

## Séparation en features et target

In [20]:
features = df.drop(columns=["averagerating", "numvotes", "weighted_score", "tconst", "primarytitle", "actor", "actress", "self", "director", "producer"])
features

,titletype,isadult,startyear,genres,seasonnumber,episodenumber,regionnumber,regionlist
0,tvEpisode,False,2020.0,"Action,Mystery",1.0,2.0,8,"[PT, IN, FR, IT, DE, JP, ES]"
1,tvSpecial,False,2016.0,Drama,NaN,NaN,2,[BD]
2,short,False,2020.0,"Music,Short",NaN,NaN,2,[US]
3,tvEpisode,False,2008.0,Drama,18.0,5.0,1,[]
4,tvEpisode,False,2020.0,"Action,Mystery",1.0,3.0,8,"[PT, IN, FR, IT, DE, JP, ES]"
...,...,...,...,...,...,...,...,...
9995,tvEpisode,False,2001.0,"Crime,Drama,Thriller",NaN,NaN,2,[MX]
9996,videoGame,False,2008.0,Adventure,NaN,NaN,2,[US]
9997,movie,False,2009.0,Drama,NaN,NaN,5,"[XWW, BR, JP, CN]"
9998,movie,True,1971.0,Adult,NaN,NaN,2,[US]


In [21]:
features.rename(columns={"self": "selfperson"}, inplace=True)

In [22]:
target = df[["weighted_score"]]
target

,weighted_score
0,6.949817
1,6.949817
2,6.949817
3,6.949817
4,6.949817
...,...
9995,6.949817
9996,6.945251
9997,6.872558
9998,6.949817


On garde 20% de test:

In [23]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

## Création de la pipeline

pour choisir les preprocessing: 

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          10000 non-null  object 
 1   titletype       10000 non-null  object 
 2   primarytitle    10000 non-null  object 
 3   isadult         10000 non-null  bool   
 4   startyear       7388 non-null   float64
 5   genres          10000 non-null  object 
 6   averagerating   1027 non-null   float64
 7   numvotes        1027 non-null   float64
 8   seasonnumber    6692 non-null   float64
 9   episodenumber   6692 non-null   float64
 10  regionnumber    10000 non-null  int64  
 11  regionlist      10000 non-null  object 
 12  actor           10000 non-null  object 
 13  self            10000 non-null  object 
 14  producer        10000 non-null  object 
 15  actress         10000 non-null  object 
 16  director        10000 non-null  object 
 17  weighted_score  10000 non-null  

On effectue différents preprocessing et encoding selon les features:

- valeurs numériques: imputation ? normalisation ? 
  - startyear 73% rempli -> on enlève les non rempli ou on impute ?
  - endyear presque non rempli -> on enlève la colonne 
  - runtimeminutes 23% rempli -> on enlève la colonne
  - seasonnumber et episodenumber 66% rempli : ceux qui n'en ont pas ne sont pas des séries -> imputation par 1
- primarytitle: tfidvectorizer
- titletype: ordinalencoder
- genres, actor, actress, self, producer, director, regionlist: multilabelbinarizer

In [25]:
features

,titletype,isadult,startyear,genres,seasonnumber,episodenumber,regionnumber,regionlist
0,tvEpisode,False,2020.0,"Action,Mystery",1.0,2.0,8,"[PT, IN, FR, IT, DE, JP, ES]"
1,tvSpecial,False,2016.0,Drama,NaN,NaN,2,[BD]
2,short,False,2020.0,"Music,Short",NaN,NaN,2,[US]
3,tvEpisode,False,2008.0,Drama,18.0,5.0,1,[]
4,tvEpisode,False,2020.0,"Action,Mystery",1.0,3.0,8,"[PT, IN, FR, IT, DE, JP, ES]"
...,...,...,...,...,...,...,...,...
9995,tvEpisode,False,2001.0,"Crime,Drama,Thriller",NaN,NaN,2,[MX]
9996,videoGame,False,2008.0,Adventure,NaN,NaN,2,[US]
9997,movie,False,2009.0,Drama,NaN,NaN,5,"[XWW, BR, JP, CN]"
9998,movie,True,1971.0,Adult,NaN,NaN,2,[US]


On définit les étapes de chaque branches

In [26]:
# Définir les pipelines
yearPipeline = Pipeline(steps=[("inputing", SimpleImputer(strategy="median")), ("scaling", StandardScaler())])
seasonEpisodeNumberPipeline = Pipeline(steps=[("inputing", SimpleImputer(strategy="constant", fill_value=1)), ("scaling", StandardScaler())])
primarytitlePipeline = Pipeline(steps=[("tfidf", TfidfVectorizer(stop_words="english"))])
titletypePipeline = Pipeline(steps=[("tfidf", OrdinalEncoder())])
genresPipeline = Pipeline(steps=[
    ('binarizer', CountVectorizer(analyzer=lambda x: set(x)))
])
actorPipeline = Pipeline(steps=[
        ('binarizer', CountVectorizer(analyzer=lambda x: set(x)))
])
actressPipeline = Pipeline(steps=[
        ('binarizer', CountVectorizer(analyzer=lambda x: set(x)))
])
selfPipeline = Pipeline(steps=[
    ('binarizer', CountVectorizer(analyzer=lambda x: set(x)))
])
producerPipeline = Pipeline(steps=[
        ('binarizer', CountVectorizer(analyzer=lambda x: set(x)))
])
directorPipeline = Pipeline(steps=[
        ('binarizer', CountVectorizer(analyzer=lambda x: set(x)))
])

On crée toutes les branches

In [36]:
preprocessing = ColumnTransformer(transformers=[
    ("year", yearPipeline, ["startyear"]),
    ("seasonEpisodeNumber", seasonEpisodeNumberPipeline, ["seasonnumber", "episodenumber"]),
    # ("primarytitle", primarytitlePipeline, "primarytitle"),
    ("titletype", titletypePipeline, ["titletype"]),
    ("genres", genresPipeline, "genres"),
    # ("actor", actorPipeline, "actor"),
    # ("selfperson", selfPipeline, "selfperson"),
    # ("actress", actressPipeline, "actress"),
    # ("producer", producerPipeline, "producer"),
    # ("director", directorPipeline, "director")
])

On Finaliser la pipeline avec le modèle

In [37]:
modelPipe = Pipeline(steps=[
    ("prep données", preprocessing),
    ("model", KNeighborsRegressor())
])

On entraine le modèle

In [38]:

# features["selfperson"] = features_train["selfperson"].apply(lambda x: ' '.join(x))


modelPipe.fit(features_train, target_train)

Pipeline(steps=[('prep données',
                 ColumnTransformer(transformers=[('year',
                                                  Pipeline(steps=[('inputing',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['startyear']),
                                                 ('seasonEpisodeNumber',
                                                  Pipeline(steps=[('inputing',
                                                                   SimpleImputer(fill_value=1,
                                                                                 strategy='constant')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['seasonnumber',
                                                   'episodenumber']),
                                                 ('titletype',
                                                  Pipeline(steps=[('tfidf',
                                                                   OrdinalEncoder())]),
                                                  ['titletype']),
                                                 ('genres',
                                                  Pipeline(steps=[('binarizer',
                                                                   CountVectorizer(analyzer=<function <lambda> at 0x0000026BD7F64C20>))]),
                                                  'genres')])),
                ('model', KNeighborsRegressor())])

On évalue le modèle

In [39]:
predict = modelPipe.predict(features_test)
predict

array([[6.95174157],
       [6.94981747],
       [6.94981747],
       ...,
       [6.94981747],
       [6.94981747],
       [6.94981747]], shape=(2000, 1))

In [40]:
# Calcul des métriques
mse = mean_squared_error(target_test, predict)
mae = mean_absolute_error(target_test, predict)
r2 = r2_score(target_test, predict)

# Affichage des métriques
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 0.009047276739462656
Mean Absolute Error (MAE): 0.012008079862809766
R-squared (R2): 0.2332777069769818
